## vanilla GAN by Goodfellow et al., 2014
##### directories that belong with this JN = MNIST and runs


##### https://www.machinecurve.com/index.php/2021/07/17/building-a-simple-vanilla-gan-with-pytorch/

In [1]:
#Imports

import os
import torch
from torch import nn
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
import matplotlib.pyplot as plt
import uuid


In [2]:
#Configurable variables 

num_epochs = 5
noise_dimension = 50
batch_size = 128
train_on_gpu = True
unique_run_id = str(uuid.uuid4())
stats_after_batch = 50
optimizer_lr = 0.0002
optimizer_betas = (0.5, 0.999)
generator_output_img_shape = 28*28*1 #784


In [3]:
#Speed ups; makes the code run faster

torch.autograd.set_detect_anomaly(False)
torch.autograd.profiler.profile(False)
torch.autograd.profiler.emit_nvtx(False)
torch.backends.cudnn.benchmark = True


In [4]:
#Generator 

class Generator(nn.Module):
    '''
    vanilla GAN generator 
    '''
    def __init__(self,):
        super().__init__()
        self.layers = nn.Sequential(
        #1st upsampling
        nn.Linear(noise_dimension, 128, bias=False),
        nn.BatchNorm1d(128, 0.8),
        nn.LeakyReLU(0.25),
        #2nd upsampling
        nn.Linear(128, 256, bias=False),
        nn.BatchNorm1d(256, 0.8),
        nn.LeakyReLU(0.25),
        #3rd upsampling
        nn.Linear(256, 512, bias=False),
        nn.BatchNorm1d(512, 0.8),
        nn.LeakyReLU(0.25),
        #Final upsampling
        nn.Linear(512, generator_output_img_shape, bias=False),
        nn.Tanh()
    )
    
    def forward(self, x):
        '''Forward pass'''
        return self.layers(x)
    

In [5]:
#Discriminator

class Discriminator(nn.Module):
    '''
    vanilla GAN discriminator
    '''
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(generator_output_img_shape, 1024),
            nn.LeakyReLU(0.25),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.25),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.25),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        '''Forward pass'''
        return self.layers(x)
    

In [6]:
#Housekeeping functions (device, directory creation, image generation, saving models, printing training progress)

def get_device():
    '''Gets device based on settings and availability'''
    return torch.device("cuda:0" if torch.cuda.is_available() and train_on_gpu else "cpu")

def make_directory_for_run():
    """ Make a directory for this training run. """
    print(f'Preparing training run {unique_run_id}')
    if not os.path.exists('./runs'):
        os.mkdir('./runs')
    os.mkdir(f'./runs/{unique_run_id}')

def generate_image(generator, epoch = 0, batch = 0, device=get_device()):
    """ Generate subplots with generated examples. """
    images = []
    noise = generate_noise(batch_size, device=device)
    generator.eval()
    images = generator(noise)
    plt.figure(figsize=(10, 10))
    for i in range(16):
    # Get image
        image = images[i]
    # Convert image back onto CPU and reshape
        image = image.cpu().detach().numpy()
        image = np.reshape(image, (28, 28))
    # Plot
        plt.subplot(4, 4, i+1)
        plt.imshow(image, cmap='gray')
        plt.axis('off')
    if not os.path.exists(f'./runs/{unique_run_id}/images'):
        os.mkdir(f'./runs/{unique_run_id}/images')
    print(f'./runs/{unique_run_id}/images/epoch{epoch}_batch{batch}.jpg') 
    plt.savefig(f'./runs/{unique_run_id}/images/epoch{epoch}_batch{batch}.jpg')

def save_models(generator, discriminator, epoch):
    """ Save models at specific point in time. """
    torch.save(generator.state_dict(), f'./runs/{unique_run_id}/generator_{epoch}.pth')
    torch.save(discriminator.state_dict(), f'./runs/{unique_run_id}/discriminator_{epoch}.pth')
    
def print_training_progress(batch, generator_loss, discriminator_loss):
    """ Print training progress. """
    print('Losses after mini-batch %5d: generator %e, discriminator %e' % (batch, generator_loss, discriminator_loss))


In [7]:
#Forwards and backwards pass

def generate_noise(number_of_images = 1, noise_dimension = noise_dimension, device=None):
    """ Generate noise for number_of_images images, with a specific noise_dimension """
    return torch.randn(number_of_images, noise_dimension, device=device)


def efficient_zero_grad(model):
    """ 
    Apply zero_grad more efficiently
    Source: https://betterprogramming.pub/how-to-make-your-pytorch-code-run-faster-93079f3c1f7b
    """
    for param in model.parameters():
        param.grad = None

        
def forward_and_backward(model, data, loss_function, targets):
    """
    Perform forward and backward pass in a generic way. Returns loss value.
    """
    outputs = model(data)
    error = loss_function(outputs, targets)
    error.backward()
    return error.item()


In [8]:
#Preparing dataset

def prepare_dataset():
    """ Prepare dataset through DataLoader """
  # Prepare MNIST dataset
    dataset = MNIST(os.getcwd(), download=True, train=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
  ]))
    
  # Batch and shuffle data with DataLoader
    trainloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    print("here's the trainloader:", trainloader)
  # Return dataset through DataLoader
    return trainloader


In [9]:
#Initialization functions (models, loss and optimizers)

def initialize_models(device = get_device()):
    """ Initialize Generator and Discriminator models """
    generator = Generator()
    discriminator = Discriminator()
  # Move models to specific device
    generator.to(device)
    discriminator.to(device)
  # Return models
    return generator, discriminator

def initialize_loss():
    """ Initialize loss function. """
    return nn.BCELoss()

def initialize_optimizers(generator, discriminator):
    """ Initialize optimizers for Generator and Discriminator. """
    generator_optimizer = torch.optim.AdamW(generator.parameters(), lr=optimizer_lr,betas=optimizer_betas)
    discriminator_optimizer = torch.optim.AdamW(discriminator.parameters(), lr=optimizer_lr,betas=optimizer_betas)
    return generator_optimizer, discriminator_optimizer


In [10]:
#Training step

def perform_train_step(generator, discriminator, real_data, loss_function, generator_optimizer, discriminator_optimizer, device = get_device()):
    """ Perform a single training step. """
  
  ## 1. PREPARATION = Set real and fake labels.
    real_label, fake_label = 1.0, 0.0
  # Get images on CPU or GPU as configured and available
  # Also set 'actual batch size', whih can be smaller than BATCH_SIZE in some cases.
    real_images = real_data[0].to(device)
    actual_batch_size = real_images.size(0)
    label = torch.full((actual_batch_size, 1), real_label, device=device)
    
  ## 2. TRAINING THE DISCRIMINATOR = Zero the gradients for discriminator
    efficient_zero_grad(discriminator)
  # Forward + backward on real images, reshaped
    real_images = real_images.view(real_images.size(0), -1)
    error_real_images = forward_and_backward(discriminator, real_images, loss_function, label)
  # Forward + backward on generated images
    noise = generate_noise(actual_batch_size, device=device)
    generated_images = generator(noise)
    label.fill_(fake_label)
    error_generated_images =forward_and_backward(discriminator, generated_images.detach(), loss_function, label)
  # Optim for discriminator
    discriminator_optimizer.step()
  
  ## 3. TRAINING THE GENERATOR = Forward + backward + optim for generator, including zero grad
    efficient_zero_grad(generator)
    label.fill_(real_label)
    error_generator = forward_and_backward(discriminator, generated_images, loss_function, label)
    generator_optimizer.step()
  
  ## 4. COMPUTING RESULTS = Compute loss values in floats for discriminator, which is joint loss.
    error_discriminator = error_real_images + error_generated_images
  # Return generator and discriminator loss so that it can be printed.

    return error_generator, error_discriminator


In [11]:
#Perform epoch

def perform_epoch(dataloader, generator, discriminator, loss_function, generator_optimizer, discriminator_optimizer, epoch):
    """ Perform a single epoch. """
    for batch_no, real_data in enumerate(dataloader, 0):
        #print(batch_no, real_data)
        # Perform training step
        generator_loss_val, discriminator_loss_val = perform_train_step(generator, discriminator, real_data, loss_function, generator_optimizer, discriminator_optimizer)
        # Print statistics and generate image after every n-th batch
        if batch_no % stats_after_batch == 0:
            #print(batch_no)
            print_training_progress(batch_no, generator_loss_val, discriminator_loss_val)
            generate_image(generator, epoch, batch_no)
        
    # Save models on epoch completion.
    save_models(generator, discriminator, epoch)
    # Clear memory after every epoch
    torch.cuda.empty_cache()


In [12]:
#Training

def train_dcgan():
    """ Train the DCGAN. """
  # Make directory for unique run
    make_directory_for_run()
  # Set fixed random number seed
    torch.manual_seed(42)
  # Get prepared dataset
    dataloader = prepare_dataset()
  # Initialize models
    generator, discriminator = initialize_models()
  # Initialize loss and optimizers
    loss_function = initialize_loss()
    generator_optimizer, discriminator_optimizer = initialize_optimizers(generator, discriminator)
  # Train the model
    for epoch in range(num_epochs):
        print(f'Starting epoch {epoch}...')
        perform_epoch(dataloader, generator, discriminator, loss_function, generator_optimizer, discriminator_optimizer, epoch)
  # Finished :-)
        print(f'Finished unique run {unique_run_id}')

    return dataloader, generator, discriminator, loss_function, generator_optimizer, discriminator_optimizer, epoch


if __name__ == '__main__':
    train_dcgan()


Preparing training run cbbd912b-7bc6-46e0-ad60-bd37c42752b4
here's the trainloader: <torch.utils.data.dataloader.DataLoader object at 0x7f14589014f0>
Starting epoch 0...


/home/stp4007/.conda/envs/ccganpytorch-new/lib/python3.9/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:189.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
/home/stp4007/.conda/envs/ccganpytorch-new/lib/python3.9/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running s

Losses after mini-batch     0: generator 6.745229e-01, discriminator 1.373596e+00
./runs/cbbd912b-7bc6-46e0-ad60-bd37c42752b4/images/epoch0_batch0.jpg
Losses after mini-batch    50: generator 1.022229e+00, discriminator 7.818463e-01
./runs/cbbd912b-7bc6-46e0-ad60-bd37c42752b4/images/epoch0_batch50.jpg
Losses after mini-batch   100: generator 1.340540e+00, discriminator 7.294370e-01
./runs/cbbd912b-7bc6-46e0-ad60-bd37c42752b4/images/epoch0_batch100.jpg
Losses after mini-batch   150: generator 1.868642e+00, discriminator 3.538273e-01
./runs/cbbd912b-7bc6-46e0-ad60-bd37c42752b4/images/epoch0_batch150.jpg
Losses after mini-batch   200: generator 1.749190e+00, discriminator 2.577002e-01
./runs/cbbd912b-7bc6-46e0-ad60-bd37c42752b4/images/epoch0_batch200.jpg
Losses after mini-batch   250: generator 2.210368e+00, discriminator 2.003060e-01
./runs/cbbd912b-7bc6-46e0-ad60-bd37c42752b4/images/epoch0_batch250.jpg
Losses after mini-batch   300: generator 3.431640e+00, discriminator 1.160016e-01
./r

<ipython-input-6-2b3e4760038c>:20: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=(10, 10))


./runs/cbbd912b-7bc6-46e0-ad60-bd37c42752b4/images/epoch2_batch0.jpg
Losses after mini-batch    50: generator 3.868990e+00, discriminator 3.352023e-01
./runs/cbbd912b-7bc6-46e0-ad60-bd37c42752b4/images/epoch2_batch50.jpg
Losses after mini-batch   100: generator 5.883999e+00, discriminator 4.193586e-01
./runs/cbbd912b-7bc6-46e0-ad60-bd37c42752b4/images/epoch2_batch100.jpg
Losses after mini-batch   150: generator 1.293555e+00, discriminator 1.094400e+00
./runs/cbbd912b-7bc6-46e0-ad60-bd37c42752b4/images/epoch2_batch150.jpg
Losses after mini-batch   200: generator 3.763147e+00, discriminator 1.695218e-01
./runs/cbbd912b-7bc6-46e0-ad60-bd37c42752b4/images/epoch2_batch200.jpg
Losses after mini-batch   250: generator 3.015695e+00, discriminator 2.875500e-01
./runs/cbbd912b-7bc6-46e0-ad60-bd37c42752b4/images/epoch2_batch250.jpg
Losses after mini-batch   300: generator 4.048510e+00, discriminator 8.717086e-01
./runs/cbbd912b-7bc6-46e0-ad60-bd37c42752b4/images/epoch2_batch300.jpg
Losses after m

In [13]:
###Test

# dataloader = prepare_dataset()
# print(type(dataloader))
# print(enumerate(dataloader))
# for i,j in enumerate(dataloader, 0):
#     print(i,j)

# generator = Generator()
# discriminator = Discriminator()
# loss_function = initialize_loss()
# generator_optimizer, discriminator_optimizer = initialize_optimizers(generator, discriminator)

# #perform_epoch(dataloader, generator, discriminator, loss_function, generator_optimizer, discriminator_optimizer, epoch)
    

here's the trainloader: <torch.utils.data.dataloader.DataLoader object at 0x7f1459175070>
<class 'torch.utils.data.dataloader.DataLoader'>
0 [tensor([[[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., 

        4, 1, 3, 6, 2, 4, 2, 9])]
10 [tensor([[[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        ...,


        [[[-1., -1., -1., 

20 [tensor([[[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        ...,


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-

29 [tensor([[[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        ...,


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-

39 [tensor([[[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        ...,


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-

...

418 [tensor([[[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        ...,


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [

428 [tensor([[[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        ...,


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [

438 [tensor([[[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        ...,


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [

448 [tensor([[[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        ...,


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [

458 [tensor([[[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        ...,


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [

#### How to load ("see") saved models

#### https://pytorch.org/tutorials/beginner/saving_loading_models.html

In [14]:
# model = Discriminator()
# model.load_state_dict(torch.load('/home/stp4007/exampleGAN/runs/53f66de5-8c5f-4a42-a2fc-7a2a8d5094d2/discriminator_4.pth'))
# model.eval()


In [15]:
# model = Discriminator()
# model.load_state_dict(torch.load('/home/stp4007/exampleGAN/runs/53f66de5-8c5f-4a42-a2fc-7a2a8d5094d2/discriminator_4.pth'))
# model.eval()
